In [98]:
# Activate multi-output in notebook
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [99]:
from os import getcwd

DATA_DIRECTORY = getcwd() + "/../data/"

DATA_2020_PATH = DATA_DIRECTORY + "2020-communes-criteres-repartition.csv"
DATA_2021_PATH = DATA_DIRECTORY + "2021-communes-criteres-repartition.csv"

In [100]:
from scripts.load_dgcl_data import load_dgcl_file

dgcl_data_2020 = load_dgcl_file(DATA_2020_PATH)
# dgcl_data_2020.keys()

dgcl_data_2021 = load_dgcl_file(DATA_2021_PATH)
# dgcl_data_2021.keys()

Loading /Users/sch/dev/gitruc/dotations-locales/data/2020-communes-criteres-repartition.csv


/Users/sch/.local/share/virtualenvs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:3: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until


Loading /Users/sch/dev/gitruc/dotations-locales/data/2021-communes-criteres-repartition.csv


/Users/sch/.local/share/virtualenvs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [101]:
filter_dgf = dgcl_data_2021["Informations générales - Population DGF Année N'"] <= 3500
filter_insee = dgcl_data_2021["Informations générales - Population INSEE Année N "] <= 3500

df_filtered = dgcl_data_2021.loc[filter_insee]

In [102]:
# dgcl_dsr_fraction_cible_2021 = dgcl_data_2021['Dotation de solidarité rurale - Cible - Montant global réparti']
# dgcl_dsu_montant_2021 = dgcl_data_2021['Dotation de solidarité urbaine - Montant total réparti']

df_2021_2020 = dgcl_data_2021.merge(
    dgcl_data_2020,
    on="Informations générales - Nom de la commune",
    # on=["Informations générales - Nom de la commune", "Informations générales - Nom de la commune"],
    suffixes=['_2021','_2020']
).fillna("rupture nom 2021-2020")


# df_2021_2020.loc[df_2021_2020["Informations générales - Nom de la commune"] == "rupture nom 2021-2020"]

In [103]:
from pandas import DataFrame
from time import strftime

result = []
colonnes = [
    'commune', 
    'population dgf', 
    'population insee'
]

for v in zip(
    df_2021_2020["Informations générales - Nom de la commune"],
    df_2021_2020["Informations générales - Population DGF Année N'_2021"],
    df_2021_2020["Informations générales - Population INSEE Année N _2021"],

):
    result.append(v)

df_result = DataFrame(result, columns=colonnes)

df_result

,commune,population dgf,population insee
0,ABERGEMENT-CLEMENCIAT,805,789
1,ABERGEMENT-DE-VAREY,307,254
2,AMBERIEU-EN-BUGEY,14771,14586
3,AMBERIEUX-EN-DOMBES,1757,1744
4,AMBLEON,130,118
...,...,...,...
43576,THIO,3580,3384
43577,TOUHO,3610,3503
43578,VOH,3750,3610
43579,YATE,2756,2694


In [104]:
# est-ce une commune de moins ou pile 3500 habitants ?

df_result["<= 3500 hab. dgf"] = df_2021_2020["Informations générales - Population DGF Année N'_2021"] <= 3500

In [105]:
def rupture_montant_une_annee(df, nom_colonne, suffixe_an1, suffixe_an2):
    return (
        df[nom_colonne+suffixe_an2] == 0
    ) * (
        df[nom_colonne+suffixe_an1] != 0
    )
    
# pour chaque commune 2021, la DSR est-elle passée à zéro cette année-là ?

df_result["dsr bourg-centre perdue"] = rupture_montant_une_annee(
    df_2021_2020, 
    'Dotation de solidarité rurale Bourg-centre - Montant global réparti', 
    '_2020', '_2021')

df_result["dsr péréquation perdue"] = rupture_montant_une_annee(
    df_2021_2020, 
    'Dotation de solidarité rurale - Péréquation - Montant global réparti (après garantie CN)', 
    '_2020', '_2021')

df_result["dsr cible perdue"] = rupture_montant_une_annee(
    df_2021_2020, 
    'Dotation de solidarité rurale - Cible - Montant global réparti', 
    '_2020', '_2021')
# df_result.loc[df_result["dsr cible perdue"] == True]

df_result.drop_duplicates(inplace = True)
df_result

,commune,population dgf,population insee,<= 3500 hab. dgf,dsr bourg-centre perdue,dsr péréquation perdue,dsr cible perdue
0,ABERGEMENT-CLEMENCIAT,805,789,True,False,False,False
1,ABERGEMENT-DE-VAREY,307,254,True,False,False,False
2,AMBERIEU-EN-BUGEY,14771,14586,False,False,False,False
3,AMBERIEUX-EN-DOMBES,1757,1744,True,False,False,False
4,AMBLEON,130,118,True,False,False,False
...,...,...,...,...,...,...,...
43576,THIO,3580,3384,False,False,False,False
43577,TOUHO,3610,3503,False,False,False,False
43578,VOH,3750,3610,False,False,False,False
43579,YATE,2756,2694,True,False,False,False


In [106]:
timestamp = strftime("%Y%m%d-%H%M%S")
df_result.to_csv(
        f'dotations_communes_evol_2021-2020_{timestamp}.csv',
        index=False,
        sep=';',
        encoding='utf-8'
        )